In [1]:
from usagrid import s3
import pandas as pd

pd.options.display.float_format = '{:20,.2f}'.format

files = s3.list_files_in_folder("usagrid","bronze/balancing_authority/Eastern/Net_generation/Net_generation_2023.arrow")[:]

dfs = [s3.read_pyarrow_df_from_s3("usagrid",f).to_pandas() for f in files[:]]


df = pd.concat(dfs)

In [8]:
def summary_stats(df):

    d = df.describe()

    d.loc["sum"] = df.sum(axis=0)
    d.loc["na_count"] = df.isna().sum(axis=0).astype(int)
    d.loc["expected_count"] = df.shape[0]

    d.loc["null_pct"] = d.loc["na_count"]/d.loc["expected_count"]

    return d

In [13]:
df = df.assign(year=df.index.year)

df_columnar = df.reset_index()\
                .pivot_table(values="value",index="period",

                columns=["year","respondent","respondent_name","type","type_name",
                        "timezone","timezone_description","value_units"]

)

df_columnar

df_columnar.pipe(summary_stats).T

count  \
year respondent respondent_name                                    type type_name      timezone timezone_description value_units                          
2023 AECI       Associated Electric Cooperative, Inc.              NG   Net generation Eastern  Eastern              megawatthours               348.00   
     AVA        Avista Corporation                                 NG   Net generation Eastern  Eastern              megawatthours               350.00   
     AVRN       Avangrid Renewables, LLC                           NG   Net generation Eastern  Eastern              megawatthours               346.00   
     AZPS       Arizona Public Service Company                     NG   Net generation Eastern  Eastern              megawatthours               353.00   
     BANC       Balancing Authority of Northern California         NG   Net generation Eastern  Eastern              megawatthours               353.00   
...                                                                                                                                                 ...   
     WACM       Western Area Power Administration - Rocky Mount... NG   Net generation Eastern  Eastern              megawatthours               342.00   
     WALC       Western Area Power Administration - Desert Sout... NG   Net generation Eastern  Eastern              megawatthours               346.00   
     WAUW       Western Area Power Administration - Upper Great... NG   Net generation Eastern  Eastern              megawatthours               352.00   
     WWA        NaturEner Wind Watch, LLC                          NG   Net generation Eastern  Eastern              megawatthours               352.00   
     YAD        Alcoa Power Generating, Inc. - Yadkin Division     NG   Net generation Eastern  Eastern              megawatthours               352.00   

                                                                                                                                                   mean  \
year respondent respondent_name                                    type type_name      timezone timezone_description value_units                          
2023 AECI       Associated Electric Cooperative, Inc.              NG   Net generation Eastern  Eastern              megawatthours            61,141.45   
     AVA        Avista Corporation                                 NG   Net generation Eastern  Eastern              megawatthours            22,548.69   
     AVRN       Avangrid Renewables, LLC                           NG   Net generation Eastern  Eastern              megawatthours            21,216.92   
     AZPS       Arizona Public Service Company                     NG   Net generation Eastern  Eastern              megawatthours            54,935.90   
     BANC       Balancing Authority of Northern California         NG   Net generation Eastern  Eastern              megawatthours            40,242.64   
...                                                                                                                                                 ...   
     WACM       Western Area Power Administration - Rocky Mount... NG   Net generation Eastern  Eastern              megawatthours           102,345.48   
     WALC       Western Area Power Administration - Desert Sout... NG   Net generation Eastern  Eastern              megawatthours            22,501.80   
     WAUW       Western Area Power Administration - Upper Great... NG   Net generation Eastern  Eastern              megawatthours             1,513.63   
     WWA        NaturEner Wind Watch, LLC                          NG   Net generation Eastern  Eastern              megawatthours             1,541.01   
     YAD        Alcoa Power Generating, Inc. - Yadkin Division     NG   Net generation Eastern  Eastern              megawatthours             2,271.14   

                                                                                                                  

In [91]:
max_col = d.loc["null_pct"].idxmax()

d[[max_col]]

respondent,MISO
respondent_name,"Midcontinent Independent System Operator, Inc."
type,D
type_name,Demand
timezone,Eastern
timezone_description,Eastern
value_units,megawatthours
count,336.00
mean,"1,784,909.38"
std,"201,585.30"
min,"1,417,717.00"


In [82]:
def filter_df(df:pd.DataFrame,level:str,value:str):

    value_index = list(df.columns.names).index(level)

    keep_cols = [col for col in df if col[value_index] == value]

    return df[keep_cols]

In [92]:
df_columnar.pipe(filter_df,level="respondent",value="MISO")

respondent,MISO
respondent_name,"Midcontinent Independent System Operator, Inc."
type,D
type_name,Demand
timezone,Eastern
timezone_description,Eastern
value_units,megawatthours
period,
2022-01-01,"1,641,398.00"
2022-01-02,"1,787,031.00"
2022-01-03,"1,957,464.00"


In [109]:
df_max = df_columnar.pipe(filter_df,level="respondent",value="MISO")

x= df_max.index

y = df_max[max_col].values

df_max["missing"] = df_max.isna()

import plotly.express as px
import plotly

fig = px.scatter(df_max,x=x,y=y,color="missing")

plotly.offline.plot(fig, filename='MISO.html')